In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from glob import glob
from tqdm import tqdm
import shutil
import nibabel as nib
from subprocess import run
import cv2

# 이미지 로드 및 전처리 함수 정의
def process_image(file_path):
    # OpenCV로 이미지 로드
    image = cv2.imread(file_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 엣지 감지 및 팽창 (Canny + Dilate)
    edges = cv2.Canny(gray, 50, 150)
    dilated = cv2.dilate(edges, np.ones((3,3), np.uint8), iterations=2)

    # 컨투어 검출
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) > 1:
        contours = sorted(contours, key=cv2.contourArea, reverse=True)
        largest_contour = contours[0]
        small_contours = contours[1:]

        # OpenCV 방식으로 마스크 생성 (PIL 대신 OpenCV 사용)
        mask = np.zeros_like(gray, dtype=np.uint8)
        cv2.fillPoly(mask, small_contours, 255)  # 작은 컨투어들만 흰색 채우기

        # NumPy 벡터 연산으로 검정색 마스크 적용 (PIL 대신 NumPy 연산 사용)
        image[mask > 0] = (0, 0, 0)  # 작은 컨투어 부분을 검정색으로 변경

    return image
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [3]:
save_path='../../data/2D_CT/'
class_list=['Ischemic','Hemorrhagic','Normal']
data_path='../../data/stroke_CT_preprocessed/'
for j in range(len(class_list)):
    dir_list=glob(data_path+class_list[j]+'/**')
    for i in tqdm(range(len(dir_list))):
        file_list=glob(dir_list[i]+'/*.png')
        file_count=len(file_list)+1
        create_dir(save_path+class_list[j])
        img=process_image(file_list[len(file_list)-20])
        Image.fromarray(img).save(save_path+class_list[j]+'/'+os.path.basename(dir_list[i])+'.png')

100%|██████████| 662/662 [00:31<00:00, 21.27it/s]


In [3]:
os.path.basename(dir_list[i])

'ANO2_1003'